In [ ]:
中文：https://twgreatdaily.com/w4bYyG0BMH2_cNUgurKB.html
英文原文：https://www.dataquest.io/blog/python-excel-xlwings-tutorial/

In [1]:
import xlwings as xw
import pandas as pd

df = pd.read_excel(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\20200324-進銷存-SF4_Vietnam_Silicom.xlsx',
                  sheet_name = 'raw')
df.sample(5)

,進銷存,2nd Item Number,Qty,LargeCat,MediumCat,Sls CD4 Desc#,Product Desc#,Year,Month,Unnamed: 9,進貨量,銷貨量,存貨量
856,存貨量,31731063101,0.0,Speaker,2.0 Speaker,SP-HF160,"RS,SP-HF160,USB,WOOD",2020,2,NaN,NaN,NaN,NaN
439,存貨量,31030114109,354.0,Mouse,Wireless Mouse,NX-7010,"RS,NX-7010,USB,TURQUOISE,BLIS!",2019,9,NaN,NaN,NaN,NaN
326,存貨量,31010116103,700.0,Mouse,Wired Mouse,DX-110,"RS,DX-110,USB,BLUE,G5",2019,3,NaN,NaN,NaN,NaN
615,存貨量,31730007401,0.0,Speaker,BT Speaker,SP-906BT PLUS-LI,"RS,SP-906BT PLUS,R2,Blue,!",2019,11,NaN,NaN,NaN,NaN
808,存貨量,31010116102,724.0,Mouse,Wired Mouse,DX-110,"RS,DX-110,USB,WHITE,G5",2020,2,NaN,NaN,NaN,NaN


In [2]:
wb = xw.Book()
sht = wb.sheets[0]
sht.name = '進銷存'  #改表名
sht.range("A1").value = df  #將df的值貼過來這個新的表(但直接貼會帶有索引)

In [5]:
sht.clear_contents()   #清除此工作表的內容(重做一次)
sht.range("A1").options(index=False).value = df  #並複製不帶索引的數據

In [ ]:
能夠知道我們的表在哪裡結束是很有用的。更具體地說，我們需要包含了數據的最後一行。為此，我們可以使用end方法和Range對象的row屬性。

不出所料，row方法會返回Range對象的row。

方法end接受一個方向(「up」(或1)、「right」(或2)、「left」(或3)、「down」(或4))作為參數，並返回另一個range對象。
它模仿Excel中常見的CTRL+Shift+箭頭動作。

In [14]:
sht.range("G9").row  #row方法會返回Range對象的row。

9

In [7]:
down_row =sht.range("A1").end('down').row  #最下面那列是excel的第幾列
down_row

1234

In [28]:
df.shape  #Dataframe中的列數

(1233, 13)

In [ ]:
並不是所有的Excel功能都可以作為一個本地xlwings特性來使用。有時我們必須找到變通辦法來做我們想做的事。
幸運的是，xlwings讓這一切變得非常簡單。來自官方文檔中「缺失的特性」一節:

解決方案:本質上，xlwings只是一個圍繞Windows平台上的pywin32和Mac平台上的appscript的靈活的包裝器。
    你可以通過調用其api屬性來訪問底層對象。底層對象將使用pywin32語法(感覺很像VBA)和appscript語法(感覺不像VBA)
    為您提供幾乎所有可以用VBA做的事情。但是除了看起來很醜之外，請記住它使您的代碼平台變成特定的(!)。
    Excel Visual Basic for Applications是對各種現有Excel對象的一個詳細解釋。

排序就是xlwings中缺少的功能之一。您可能已經注意到，那些記錄是從最近到最遠的抽籤順序排列的。在接下來的幾個步驟中，我們將反轉順序。

對象ws.range(「A2:N{row}」.format(row=last_row))是一個Range對象。將api屬性追加給它之後，
會生成一個VBA Range對象，該對象反過來又可以訪問它的VBA特性。

我們將使用這個VBA對象的Sort屬性。在它的最簡單的應用程式中，Sort接受兩個參數:
對表排序所使用的列(作為一個VBA Range對象)和排序類型(無論希望按升序還是降序排序)。
第二個參數的參數文檔可以在這裡看到。我們將按升序進行排序。

### 排序

In [120]:
sht.range('A2').options(expand='table').api.Sort(Key1=sht.range("C:C").api, Order2=0,  Header="xlNo")  
#依照C欄排序
#Order1说明：1代表升序，2代表降序

True

In [69]:
sht.range('A2').options(expand='table')

<Range [Book6]進銷存!$A$2>

In [117]:

sht.range('C2').api.Sort(Key1=sht.range("D2").api, Order1=2,
                         Key2=sht.range("E2").api, Order2=2,
                         Key3=sht.range("C2").api, Order3=2,
                         Header="xlYes") 
#依照C欄排序
#Order1说明：1代表升序，2代表降序

True

In [121]:
#排序
sht.range('C2').api.Sort(Key1=sht.range("C2").api, Order1=2)

True

In [107]:
sht.range('A2:I1234').api.Sort(Key1=sht.range("C2").api, Order1=2,
                         Key2=sht.range("D2").api, Order2=1,
                         Key3=sht.range("E2").api, Order3=1,
                         Header="xlYes") 

True

In [ ]:
DA

# 公式下拉
在第一個單元格中插入公式後，在常規的Excel工作流中，第二個特性是通過表的末尾自動填充其餘單元格。

### 法一：用迴圈
缺點：1.要到哪一列得自己設定 2.公式不會留著會被貼上值

In [21]:
import xlwings as xw

wb = xw.Book(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\Book1.xlsx')
sheet1 = wb.sheets['Sheet1']

for i in range(3, 100):   
    price_today = sheet1.cells(i, "B").value  # 到 B 欄截取今日收盤價
    price_yesterday = sheet1.cells(i-1, "B").value  # 到 B 欄截取迭代到的上一個 row, 也就是昨日收盤價
    daily_return = (price_today - price_yesterday) / price_yesterday  # 算出該筆資料的日報酬率了   
    sheet1.cells(i, "C").value = daily_return   # 最後再將算出的日報酬率寫入同一列，欄為 C 的儲存格

### 法二：Autofill
在第一個單元格中插入公式後，在常規的Excel工作流中，第二個特性是通過表的末尾自動填充其餘單元格。
Autofill是VBA Range對象的一個方法。它接受兩個參數，即一個將目標單元格作為參數的VBA Range對象和填充類型。
我們感興趣的是枚舉為0的默認值。

In [7]:
import xlwings as xw

wb = xw.Book(r'C:\Users\11004076\Documents\Python Scripts\4_Automation\Book1.xlsx')
sheet2 = wb.sheets['Sheet2']

sheet2.range("C3").value = "=(B3-B2)/B2"

sheet2.range("C3").api.AutoFill(
    sheet2.range("C3:C{row}".format(row=sheet2.range("B1").end('down').row)).api,
    0
)

True

# 上色
若當天報酬率是大於0，就把底色換成紅色；
相對的，若當天報酬率是小於0，就把底色換成綠色：

In [23]:
for i in range(3, 100):
    daily_return = sheet1.cells(i, 'C').value
    
    if daily_return > 0:  # 若該筆資料的日報酬率是上漲的 (大於 0)  
        sheet1.cells(i, 'C').color = (255, 0, 0)  # 就把該儲存格的底色換成紅色   
    elif daily_return < 0:  # 若該筆資料的日報酬率是下跌的 (小於 0)
        sheet1.cells(i, 'C').color = (0, 255 , 0)  # 就把該儲存格的底色換成綠色

In [52]:
import pandas as pd
import xlwings as xw
df = pd.read_csv('euromillions.csv')
df.sample(5)

,No.,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,L2,Jackpot,Wins
772,480,Fri,27,Apr,2012,36,43,30,27,20,1,6,36231900,1
240,1012,Fri,2,Jun,2017,8,42,33,24,10,9,3,134808762,1
1058,194,Fri,26,Oct,2007,20,7,38,40,43,8,1,19424115,0
740,512,Fri,17,Aug,2012,19,25,33,44,28,8,2,21173767,0
410,842,Fri,16,Oct,2015,28,43,7,48,29,3,10,11035500,0


In [53]:
wb = xw.Book()
ws = wb.sheets["Sheet1"]
ws.name = "EuroMillions"
ws.range("A1").options(index=False).value = df

In [54]:
last_row = ws.range(1,1).end('down').row
print("The last row is {row}.".format(row=last_row))
print("The DataFrame df has {rows} rows.".format(rows=df.shape[0]))

The last row is 1253.
The DataFrame df has 1252 rows.


In [55]:
ws.range(
"A2:N{row}".format(row=last_row)
).api.Sort(Key1=ws.range("A:A").api, Order1=1)

True

In [56]:
ws.range("O1").value = "Date"
ws.range("O2").value = "=C2&D2&RIGHT(E2, 2)"

In [57]:
ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
0
)

True

In [ ]:
我們還可以使用所需填充類型的命名形式。為此，我們需要從模塊xlwings.constants中檢索它，該模塊包含大多數VBA屬性的枚舉參數的命名版本。
您可以經常通過列印dir(xlwings.constants)來檢查可用的屬性。

(如果您不熟悉它，dir是一個本地Python函數，可以接受多種參數(模塊、類和常規對象(如列表和字符串))。
 例如，如果您列印dir(some_list)，它將為您輸出您可以通過列表使用的所有方法和屬性。

我們在上面所做的也可以通過下面的代碼片段來實現。

In [58]:
from xlwings.constants import AutoFillType
ws.range("O2").api.AutoFill(
ws.range("O2:O{row}".format(row=last_row)).api,
    AutoFillType.xlFillDefault
)

True

In [ ]:
由於我們將經常使用這個操作，我們將創建一個應用默認填充的函數，給定:

一個工作表
一個表示工作表中單元格的字符串
一個需要填充的末尾行。
為此，我們將引入一個名為get_address的新的Range方法。它接受四個布爾參數並返回一個字符串，
該字符串標識不同詳細級別的範圍。下面是這個方法的一個很有啟發性的說明。


In [43]:
for arg4 in (0, 1):
for arg3 in (0,1):
for arg2 in (0,1):
for arg1 in (0,1):
print(ws.range("O2").get_address(arg1, arg2, arg3, arg4))

IndentationError: expected an indented block (<ipython-input-43-225b191b5f1a>, line 2)

![Image of Yaktocat](https://twgreatdaily.com/images/elastic/-4b/-4bzyG0BMH2_cNUgg98Z.jpg)

In [ ]:
現在我們來定義函數。

In [59]:
def autofill(worksheet, cell, last_row):
   rg_cell = worksheet.range(cell)
   to_fill = "{col}{top_row}:{col}{last_row}".format(
      col=rg_cell.get_address(0,0)[0],
      top_row=rg_cell.row,
      last_row=last_row
   )
   rg_cell.api.Autofill(worksheet.range(to_fill).api, 0)

In [ ]:
為了避免Excel進行不必要的計算，我們將使用硬編碼的值替換剛才插入到列O中的公式。
在此之前，讓我們花點時間思考一下當Range是一個數組時，Range.value是什麼類型的Python對象。

In [60]:
print(type(ws.range("O2:O{row}".format(row=last_row)).value))

<class 'list'>


In [ ]:
它是一個列表。我們來看它的前十個元素。

In [61]:
print(ws.range('O2:O{row}'.format(row=last_row)).value[:10])

['13Feb04', '20Feb04', '27Feb04', '5Mar04', '12Mar04', '19Mar04', '26Mar04', '2Apr04', '9Apr04', '16Apr04']


In [ ]:
如果我們將這個列表插入到任何範圍中，它將會水平放置值，這不是我們想要的。
為了將它們垂直放置，我們需要使用Range對象的options方法，並將transpose=True選項作為一個參數，如下所示:

In [62]:
#貼上值
ws.range('O2').options(transpose=True).value\
= ws.range('O2:O{row}'.format(row=last_row)).value

In [ ]:
現在我們可以刪除列C到E。

In [63]:
ws.range('C:E').api.Delete()

True

In [ ]:
EuroMillions格式多年來經歷了一些溫和的修改，最後一次修改是在2016年9月24日。

從2016年9月24日起，幸運星的數量從11個數字變為12個數字。為了進行有意義的分析，我們只考慮在最後修改之後進行的抽獎。
下一個代碼片段將查找修改之前的最後一次抽獎，並將其命名為to_delete。

In [67]:
ws.range('L2:L{}'.format(last_row))

<Range [Book4]EuroMillions!$L$2:$L$1253>

In [70]:
import datetime

for day in ws.range('L2:L{}'.format(last_row)):  # checks if day is not prior to the change of the rules
    if day.value <= datetime.datetime(2016, 9, 24, 0, 0):
        # since day is past the modification date,
        # the row we want is the previous one, hence minus 1
        to_delete = int(day.get_address(0, 0)[1:])-1
        # leave the for cycle
        break

In [ ]:
我們現在可以刪除從第一次抽獎到to_delete的每一行。

In [51]:
to_delete

1

In [50]:
ws.range('2:{}'.format(to_delete)).api.Delete()

True

In [ ]:
這是我們目前的情況，此時:

In [ ]:
在完成數據準備之後，我們現在將格式化這個表。我們首先將第一行的字體設置為粗體。

In [72]:
ws.range('1:1').api.Font.Bold = True

In [ ]:
我們可以按此操作將「Jackpot」列以百萬為單位進行格式化。請注意，下面的字符串格式取決於您機器的本地區域設置。
如果您的格式看起來很奇怪，試著用圓點替換逗號。更多關於Excel自定義格式的內容，請查看這裡。
（地址：https://exceljet.net/custom-number-formats ）

In [75]:
#自訂數值格式
ws.range('J:J').number_format = "#!.0,萬"

In [ ]:
作為後續工作的輔助步驟，我們將找到與最後一列數據對應的字母。

In [77]:
last_column = ws.range(1,1).end('right').get_address(0,0)[0]
last_column

'L'

In [ ]:
現在，我們在header單元格的底部添加邊框。與我們所做的類似，我們將使用api屬性。
此外，我們還將需要Range對象的Border屬性、邊框對齊枚舉和邊框樣式。
我們將在header單元格的底部設置一個雙邊邊框(行樣式-4119，對齊方式9)。

In [85]:
ws.range('A1:{}1'.format(last_column)).api.Borders(9).LineStyle = -4119

In [ ]:
現在讓我們對行和列進行自動調整。

In [81]:
ws.autofit()

In [ ]:
哎呦!這看起來有點壓扁了，讓我們設置所有列的寬度為J列的寬度，這似乎是最大的值。
這裡（https://docs.microsoft.com/en-us/office/vba/api/excel.range.columnwidth ） 是我們下面正在使用的ColumnWidth的文檔。

In [82]:
ws.range('A:L').api.ColumnWidth = ws.range('J:J').api.ColumnWidth

In [ ]:
那樣看起來更好一點。這個工作表我們就處理完成了!

讓我們add一個名為Frequencies的新空白工作表，並將其分配給Python變量frequencies。

In [86]:
wb.sheets.add('Frequencies')
frequencies = wb.sheets['Frequencies']

In [ ]:
我們將在這個工作表中填入我們剛剛在EuroMillions工作表中處理過的數據集中每個數字和每個幸運星的絕對頻率。

In [87]:
# add a header for the numbers
frequencies.range('A1').value = 'Number'
# populate the fifty cells immediately below with the numbers 1 through 50
# since we're starting from the second row, we'll want to substract 1 from the row
frequencies.range('A2:A51').value = '=ROW()-1'  #公式:當前列數-1→製造序列

In [ ]:
下面，我們將為單元格B1中的頻率插入一個header，在單元格B2中，我們將輸入一個公式，計算A2中的值在C2:G201範圍內出現的次數。
換句話說，我們將計算1在N1-N5列中出現的次數。在此之後，我們將自動填充B列上的其餘單元格，以對它們各自的行執行相同的操作。

In [89]:
# add a header for the frequencies
frequencies.range('B1').value = 'Frequency'
# insert on B2 the result of a standard Excel formula
frequencies.range('B2').value = '=COUNTIF(Euromillions!$C$2:$G$201,Frequencies!A2)'
autofill(frequencies, 'B2', 51)  #前面的def function

In [ ]:
我們對幸運星也執行同樣的操作:

In [91]:
frequencies.range('D1').value = 'Lucky Star'
frequencies.range('E1').value = 'Frequency'
frequencies.range('D2:D13').value = '=ROW()-1'
frequencies.range('E2').value =\   #(\):可以換行繼續輸入
'=COUNTIF(EuroMillions!$H$2:$I$201,Frequencies!D2)'
autofill(frequencies, 'E2', 13)
frequencies.autofit()

In [ ]:
此時，我們的新工作表看起來應該像這樣：

In [ ]:
我們正在接近我們的目標。讓我們創建一個名為Graphs的工作表。

In [99]:
wb.sheets.add('Graphs')
graphs = wb.sheets['Graphs']

In [ ]:
現在我們將創建一個Chart對象。這只會產生一個空白的白框，但是不要擔心!我們稍後會用這個方框來圖表化我們的數據。

In [117]:
nr_freq = xw.Chart()

In [ ]:
我們可以像name工作表一樣來命名圖表。方法set_source_data允許我們通過傳入一個範圍對象來定義我們圖表的數據源。

In [118]:
nr_freq.name = 'Number Frequencies'
nr_freq.set_source_data(frequencies.range('Frequencies!B1:B51'))

In [ ]:
Excel將嘗試猜測x軸應該是什麼，但我們可以使用VBA Chart方法FullSeriesCollection來強制讓它成為我們在Frequencies上創建的數字。
我們可以通過使用nr_freq.api索引1的對象來編輯圖表:

In [124]:
nr_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!A2:A51'

com_error: (-2147352570, '不明的名稱。', None, None)

In [ ]:
Excel非常擅長猜測用戶想要什麼樣的圖表，但是為了防止它猜錯，我們將強制它成為柱狀圖。
這裡列出了各種類型的圖表。唉，將這些圖表與chart_type屬性的可能值連接起來的唯一文檔就是其原始碼本身。

In [120]:
nr_freq.chart_type = 'column_clustered'

In [ ]:
現在我們將定義圖表的高度和寬度。度量單位為points。

In [121]:
nr_freq.height = 250
nr_freq.width = 750

In [ ]:
此時，我們應該會看到:

In [ ]:
使用SetElement方法和參數2一起設置圖表上方的標題。請在這裡查看其它參數。
(地址;https://docs.microsoft.com/en-us/dotnet/api/microsoft.office.core.msochartelementtype?view=office-pia )

In [122]:
nr_freq.api[1].SetElement(2)  # Place chart title at the top
nr_freq.api[1].ChartTitle.Text = 'Number Frequencies'

In [ ]:
我們來添加最後的修飾。我們使用HasLegend屬性刪除圖例。

In [125]:
nr_freq.api[1].HasLegend = 0

In [ ]:
我們將xlCategory類別作為參數1傳入給Axes方法，並將TickLabelSpacing屬性設置為1，這確保了圖表軸上的每個元素都被顯示。

In [126]:
nr_freq.api[1].Axes(1).TickLabelSpacing = 1

In [ ]:
為了完成對這個圖表的格式化，我們通過將Line對象的Visible屬性設置為0來刪除邊線。

In [131]:
graphs.shapes.api('Number Frequencies').Line.Visible = 0

In [ ]:
下面我們對幸運星也做了幾乎同樣的事情。

In [133]:
ls_freq = xw.Chart()
ls_freq.top = 250
ls_freq.name = 'Lucky Star Frequencies'
ls_freq.set_source_data(frequencies.range('Frequencies!E1:E13'))
#ls_freq.api[1].FullSeriesCollection(1).XValues = '=Frequencies!D2:D13'   #這串會Error
ls_freq.chart_type = 'column_clustered'
ls_freq.height = 250
ls_freq.width = 750
ls_freq.api[1].SetElement(2)
ls_freq.api[1].ChartTitle.Text = 'Lucky Star Frequencies'
ls_freq.api[1].HasLegend = 0
ls_freq.api[1].Axes(1).TickLabelSpacing = 1
graphs.shapes.api('Lucky Star Frequencies').Line.Visible = 0

In [ ]:
最後，我們創建了一個顯示jackpot演變的時間序列圖。

In [138]:
jackpot = xw.Chart()
jackpot.top = 500
jackpot.name = 'Jackpot'
last_row = ws.range(1,1).end('down').row
jackpot.set_source_data(ws.range('Euromillions!J2:J{}'.format(last_row)))
#jackpot.api[1].FullSeriesCollection(1).XValues= 'Euromillions!L2:L{}'.format(last_row)   #這串會Error
jackpot.chart_type = 'line'
jackpot.height = 250
jackpot.width = 750
jackpot.api[1].SetElement(2)
jackpot.api[1].ChartTitle.Text = 'Jackpot'
jackpot.api[1].HasLegend = 0
graphs.shapes.api('Jackpot').Line.Visible = 0

In [ ]:
我們通過將TickLabels的NumberForma屬性設置為期望的外觀來修復垂直軸「labels」的格式。

In [148]:
jackpot.api[1].Axes(2).TickLabels.NumberFormat = "#,萬"